In [9]:
import glob
import os
import re

from PIL import Image
import torch
from torchvision import transforms
import warnings
warnings.filterwarnings('ignore')

Refer for code: https://pytorch.org/hub/pytorch_vision_googlenet/

In [2]:
# Download ImageNet Labels
# !wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

In [8]:
# Read ImageNet categories
with open('./imagenet_classes.txt', 'r') as f:
    categories = [s.strip() for s in f.readlines()]

print(*categories, sep = '\n')

tench
goldfish
great white shark
tiger shark
hammerhead
electric ray
stingray
cock
hen
ostrich
brambling
goldfinch
house finch
junco
indigo bunting
robin
bulbul
jay
magpie
chickadee
water ouzel
kite
bald eagle
vulture
great grey owl
European fire salamander
common newt
eft
spotted salamander
axolotl
bullfrog
tree frog
tailed frog
loggerhead
leatherback turtle
mud turtle
terrapin
box turtle
banded gecko
common iguana
American chameleon
whiptail
agama
frilled lizard
alligator lizard
Gila monster
green lizard
African chameleon
Komodo dragon
African crocodile
American alligator
triceratops
thunder snake
ringneck snake
hognose snake
green snake
king snake
garter snake
water snake
vine snake
night snake
boa constrictor
rock python
Indian cobra
green mamba
sea snake
horned viper
diamondback
sidewinder
trilobite
harvestman
scorpion
black and gold garden spider
barn spider
garden spider
black widow
tarantula
wolf spider
tick
centipede
black grouse
ptarmigan
ruffed grouse
prairie chicken
peacock

In [5]:
# Path to all image files (jpg/jpeg)
data_path = './data/*.j*'

# Creating a mapping in the form of filename: path
basename_path_mapping = {}

for file in glob.glob(data_path):
    basename_path_mapping[os.path.basename(file).split('.')[0]] = file
    
#print(basename_path_mapping)

In [10]:
# Initializing the VGG model
model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)
model.eval()

Using cache found in C:\Users\thara/.cache\torch\hub\pytorch_vision_v0.10.0


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [15]:
# Traversing each image and getting top 15 probabilities using VGG16
for label, path in basename_path_mapping.items():
    input_image = Image.open(path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
    # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
    # print(output[0])
    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    # print(probabilities)
    print('-----------------')
    print(label)
    print('-----------------')
    top15_prob, top15_catid = torch.topk(probabilities, 15)
    for i in range(top15_prob.size(0)):
        print(categories[top15_catid[i]], round(top15_prob[i].item(),5))
    print()
    print()

-----------------
aquariumfish1
-----------------
goldfish 0.74469
ladybug 0.04406
hummingbird 0.00806
eft 0.00617
cock 0.00529
agama 0.00511
ocarina 0.00488
hip 0.00343
green mamba 0.00337
puffer 0.0022
hen 0.00192
Yorkshire terrier 0.0019
rock beauty 0.0019
handkerchief 0.00182
tree frog 0.00173


-----------------
aquariumfish2
-----------------
anemone fish 0.89731
sea anemone 0.02564
coral reef 0.02208
rock beauty 0.00257
sea slug 0.00214
goldfish 0.00126
coral fungus 0.00115
daisy 0.001
lionfish 0.00087
shower curtain 0.00075
scuba diver 0.00068
brain coral 0.00061
digital clock 0.00046
sea urchin 0.00046
piggy bank 0.00045


-----------------
aquariumfish3
-----------------
snail 0.0854
hummingbird 0.05166
gar 0.02059
cricket 0.01955
ruffed grouse 0.01795
goldfish 0.01738
puffer 0.01566
weevil 0.01469
prairie chicken 0.01387
bittern 0.01189
robin 0.01115
slug 0.01023
tench 0.00937
hermit crab 0.00823
bustard 0.00792


-----------------
aquariumfish4
-----------------
coral reef 

In [16]:
# Create a mapping of class items to super class
classes = {
    'dinosaur': [51],
    'couch': [831],
    'mushroom': [947, 992, 993, 994, 995, 996, 997],
    'crab': [118, 119, 120, 121, 125],
    'aquariumfish': [1, 5, 107, 123, 124, 327, 395, 396, 397],
    'rose': []
}

# Displaying the names of the classes in imagenet
for key, value in classes.items():
    print('{}: {}'.format(key, [categories[idx] for idx in value]))

dinosaur: ['triceratops']
couch: ['studio couch']
mushroom: ['mushroom', 'agaric', 'gyromitra', 'stinkhorn', 'earthstar', 'hen-of-the-woods', 'bolete']
crab: ['Dungeness crab', 'rock crab', 'fiddler crab', 'king crab', 'hermit crab']
aquariumfish: ['goldfish', 'electric ray', 'jellyfish', 'spiny lobster', 'crayfish', 'starfish', 'gar', 'lionfish', 'puffer']
rose: []


In [17]:
# Traversing each image and getting superclass probability
for label, path in basename_path_mapping.items():
    input_image = Image.open(path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
    # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
    # print(output[0])
    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    probabilities = torch.nn.functional.softmax(output[0], dim=0)

    # Getting the class probability for the image
    image_class = re.sub(r'\d', '', label)
    print('{}: {}'.format(label, sum([probabilities[idx] for idx in classes[image_class]])))

aquariumfish1: 0.7484462857246399
aquariumfish2: 0.0029901135712862015
aquariumfish3: 0.06563643366098404
aquariumfish4: 0.14952820539474487
aquariumfish5: 0.02272125519812107
aquariumfish6: 0.9889709949493408
aquariumfish7: 0.01234479621052742
aquariumfish8: 0.0463944673538208
couch1: 0.9136160612106323
couch2: 0.9760825037956238
couch3: 0.8396257162094116
couch4: 0.8400134444236755
couch5: 0.14603567123413086
couch6: 0.0002772026928141713
couch7: 0.13088147342205048
couch8: 0.49557387828826904
crab1: 0.9991505146026611
crab2: 0.8653722405433655
crab3: 0.9971087574958801
crab4: 0.9098654985427856
crab5: 0.8438724875450134
crab6: 0.9959153532981873
crab7: 0.010993258096277714
crab8: 0.9767907857894897
dinosaur1: 0.5553574562072754
dinosaur2: 0.3305862247943878
dinosaur3: 0.3920108377933502
dinosaur4: 0.041128307580947876
dinosaur5: 0.11340274661779404
dinosaur6: 0.13192211091518402
dinosaur7: 0.9440904259681702
dinosaur8: 0.4505298435688019
mushroom1: 0.704749345779419
mushroom2: 0.183